In [1]:
import glob
import subprocess

- sample_in.txt
- sample_out.txt
- statement.txt
- full_in.txt

In [2]:
# reset

import shutil
shutil.rmtree("execution_sample_out", ignore_errors=True)
shutil.rmtree("execution_full_out", ignore_errors=True)
shutil.rmtree("execution_code", ignore_errors=True)

In [3]:
# TODO - change this during contest
contest_folder = 'practice'
password = 'w3ak_password_for_practice'

subprocess.run(
    [
        '7z', 'x', f'{contest_folder}.zip',
        '-pw3ak_password_for_practice',
        f'-o{contest_folder}',
        '-y'
    ]
)


7-Zip [64] 17.05 : Copyright (c) 1999-2021 Igor Pavlov : 2017-08-28
p7zip Version 17.05 (locale=utf8,Utf16=on,HugeFiles=on,64 bits,8 CPUs x64)

Scanning the drive for archives:
1 file, 40894256 bytes (39 MiB)

Extracting archive: practice.zip
--
Path = practice.zip
Type = zip
Physical Size = 40894256

Everything is Ok

Files: 23
Size:       84913192
Compressed: 40894256


CompletedProcess(args=['7z', 'x', 'practice.zip', '-pw3ak_password_for_practice', '-opractice', '-y'], returncode=0)

In [4]:
import glob
import os

folders = glob.glob("./practice/*/")  # only select folders

problem_names = [os.path.basename(os.path.normpath(folder)) for folder in folders]

problem_names

['Line of Delivery (Part 1)',
 'Line of Delivery (Part 2)',
 'Line by Line',
 'Fall in Line',
 'Walk the Line']

In [5]:
thread_per_question = 1

In [6]:
contest_folders = [contest_folder] * len(problem_names) * thread_per_question
passwords = [password] * len(problem_names) * thread_per_question
problem_names = problem_names * thread_per_question
solution_ids = [f"{i:03}" for i in range(thread_per_question)] * len(problem_names)

In [7]:
import pandas as pd

df = pd.DataFrame(
    {
        "contest_folder": contest_folders,
        "password": passwords,
        "problem_name": problem_names,
        "solution_id": solution_ids,
    }
)

In [8]:
# solve(contest_folder, password, problem_name, "001")

In [9]:
from worker import solve

In [ ]:
import concurrent.futures

with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
    # run in parallel
    results = executor.map(solve, df["contest_folder"], df["password"], df["problem_name"], df["solution_id"])
    results = list(results)